In [1]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-23 23:10:33--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-23 23:10:33 (33.5 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [4]:
import minsearch

In [5]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-23 23:13:14--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.03s   

2024-06-23 23:13:15 (18.2 MB/s) - ‘documents.json’ saved [658332/658332]



In [5]:
import json

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
	
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [10]:
q = 'the course has already started, can I still enroll?'

In [11]:
index.fit(documents)

In [12]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict= {'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [13]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [14]:
from openai import OpenAI

In [16]:
client = OpenAI()

In [17]:
	
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It's best to contact the course instructor or the institution offering the course to inquire about late enrollment options. Some courses may allow you to join after the start date, while others may have strict deadlines. Be prepared to catch up on any missed material if you are allowed to enroll late."

In [66]:
	
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )

    return results

In [67]:
	
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [68]:
	
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [69]:
	
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [70]:
rag(query)

'To run Kafka in the context of your project, follow these instructions based on the role:\n\n### Java Kafka\n1. **Navigate to the project directory** where your Java files are located.\n2. **Run the following command** in the terminal:\n   ```sh\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n   Ensure you replace `<jar_name>` with the actual name of your JAR file.\n\n### Python Kafka\n1. **Create and activate a virtual environment**:\n   ```sh\n   python -m venv env\n   source env/bin/activate  # For macOS and Linux\n   env\\Scripts\\activate     # For Windows\n   ```\n2. **Install the required packages**:\n   ```sh\n   pip install -r ../requirements.txt\n   ```\n3. **Run your Python producer/consumer scripts** in the virtual environment.\n\n### Docker Containers\nTo ensure your Kafka broker is running, follow these steps:\n1. **Check the Docker containers**:\n   ```sh\n   docker ps\n   ```\n2. If the Kafka broker container

In [45]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll even if the course has already started. However, you won't be able to submit some of the homeworks. To be eligible for a certificate, you need to submit 2 out of the 3 course projects and review 3 peers' projects by the deadline. This means that if you join the course late but manage to work on two projects, you will still be eligible for a certificate. Make sure to keep an eye on the deadlines for the final projects to avoid leaving everything to the last minute."

In [46]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [47]:
from elasticsearch import Elasticsearch

In [48]:
es_client = Elasticsearch('http://localhost:9200')

In [30]:
	
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [49]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [50]:
from tqdm.auto import tqdm

In [51]:
	
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:16<00:00, 57.68it/s]


In [52]:
query = 'I just disovered the course. Can I still join it?'

In [77]:
	
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [90]:
	
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer, len(prompt)

In [91]:
rag(query)

("To annotate a graph in Matplotlib, you can use the `annotate` method, which allows you to specify an X,Y point and add an annotation with an arrow and text. Here is an example that demonstrates how to show an arrow pointing to the x,y point of the optimal threshold, along with the corresponding text annotation:\n\n```python\nplt.annotate(\n    f'Optimal Threshold: {optimal_threshold:.2f}\\nOptimal F1 Score: {optimal_f1_score:.2f}',\n    xy=(optimal_threshold, optimal_f1_score),\n    xytext=(0.3, 0.5),\n    textcoords='axes fraction',\n    arrowprops=dict(facecolor='black', shrink=0.05)\n)\n```\n\nIn this code snippet:\n- `xy` defines the point to annotate.\n- `xytext` defines the position of the text.\n- `textcoords='axes fraction'` specifies that the text coordinates are fractions of the axes dimensions.\n- `arrowprops` allows you to customize the appearance of the arrow.",
 2815)

# Question 4


In [92]:
query = 'How do I debug a docker container?'
rag(query)

('To debug a Docker container, you can launch the container image in interactive mode and override the entrypoint so that it starts with a bash command. Here are the steps:\n\n1. Launch the container interactively with bash:\n   ```sh\n   docker run -it --entrypoint bash <image>\n   ```\n\n2. If the container is already running, execute a bash command in the specific container by following these steps:\n   a. List the running containers to find the container ID:\n      ```sh\n      docker ps\n      ```\n   b. Execute a bash shell on the specific container using its container ID:\n      ```sh\n      docker exec -it <container-id> bash\n      ```\n\nThese commands will allow you to interact with the container and debug it as needed.\n\n(Marcos MJD)',
 2452)

In [93]:
query = 'How do I copy files from a different folder into docker container’s working directory?'
rag(query)

('To copy files from a different folder into a Docker container’s working directory, you can use the Docker `COPY` command in your Dockerfile. The basic syntax for copying specific files is as follows:\n\n```Dockerfile\nCOPY ["src/predict.py", "models/xgb_model.bin", "./"]\n```\n\nThis command indicates to Docker to copy `predict.py` from the `src` folder and `xgb_model.bin` from the `models` folder into the working directory of the container.',
 2642)

In [98]:
query = 'How do Lambda container images work?'
rag(query)

('To understand how Lambda container images work and how Lambda functions are initialized, you can refer to the following detailed documentation from AWS:\n\n1. [Creating and working with Lambda container images](https://docs.aws.amazon.com/lambda/latest/dg/images-create.html)\n2. [Using the Lambda runtime API](https://docs.aws.amazon.com/lambda/latest/dg/runtimes-api.html)\n\nThese resources provide in-depth information on the setup, creation, and initialization processes for Lambda container images.',
 2046)

In [97]:
query = 'How can I annotate a graph?'
rag(query)

('To annotate a graph, you can use Matplotlib\'s `annotate` method. This method allows you to provide an X,Y point and annotate it with an arrow and text. For instance, if you want to show an arrow pointing to a specific point with an annotation, you can do the following:\n\n```python\nimport matplotlib.pyplot as plt\n\n# Example values for the annotation, replace with your actual values\noptimal_threshold = 0.5\noptimal_f1_score = 0.85\n\nplt.annotate(\n    f\'Optimal Threshold: {optimal_threshold:.2f}\\nOptimal F1 Score: {optimal_f1_score:.2f}\',\n    xy=(optimal_threshold, optimal_f1_score),\n    xytext=(0.3, 0.5),\n    textcoords=\'axes fraction\',\n    arrowprops=dict(facecolor=\'black\', shrink=0.05)\n)\n```\n\nThis code will display an annotation with the text "Optimal Threshold: 0.50\\nOptimal F1 Score: 0.85," pointing to the coordinates (0.5, 0.85) on the graph. Adjust the `xy` and `xytext` parameters as needed for your specific graph and data points.',
 2815)